Парсинг вакансий с hh.ru

In [185]:
import pandas as pd
import requests
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
from time import sleep

Стартовый URL и заголовки, поиск по вакансии 'Data science'

In [186]:
url = 'https://krasnoyarsk.hh.ru/search/vacancy?area=1&search_field=name&search_field=company_name&search_field=description&enable_snippets=true&text=Data+science&ored_clusters=true'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
          }

Основная функция парсера.
Параметры - URL страница поиска, заголовки, список словарей с поисковыми полями

Cтруктура словаря:
dict_vacancy = {       
                       'name_vacancy': name_vacancy,                                   # имя вакансии
                       'min_vacancy_compensation': min_vacancy_compensation,           # зарплата от
                       'max_vacancy_compensation': max_vacancy_compensation,           # зарплата до
                       'currency_vacancy_compensation': currency_vacancy_compensation, # валюта зарплаты
                       'link_vacancy': link_vacancy,                                   # ссылка на вакансию
                       'vacancy_employer': vacancy_employer,                           # работодатель
                       'vacancy_address': vacancy_address                              # адрес работодателя
               }

In [187]:
def parser_vacancy_hhru(url_get, headers_get, list_dict_vacancies = []):
    sleep(3)
    response = requests.get(url=url_get, headers=headers_get)
    if response.status_code != 200:
        print('stop parsing!')
        return list_dict_vacancies
    print('parsing start!!!')
    soup = bs(response.content, 'html.parser')
    list_dict_vacancies = search_vacancies_soup(soup, list_dict_vacancies)
    link_pager_next = soup.find('a', {'data-qa': 'pager-next'})
    if link_pager_next:
        url_next_page = 'https://krasnoyarsk.hh.ru' + link_pager_next['href']
    else:
        print('parsing stop!!!')
        return list_dict_vacancies
    parser_vacancy_hhru(url_get = url_next_page, headers_get=headers_get, list_dict_vacancies = list_dict_vacancies)
    return list_dict_vacancies

Вспомогательная функция преобразования строки зарплаты в список цифр зарплаты

In [188]:
def str_to_digit(text):
    text_digits = ''.join(letter for letter in text if letter in ' \t0123456789')
    return text_digits.split()

Вспомогательная функция преобразования строки зарплаты в строку с валютой зарплаты

In [189]:
def currency(text):
    return text[-4:].split()[0]

На вход функция принимает поисковую строку по зарплате, возвращает минимальную зарплату, максимальную зарплату, валюту зарплаты

In [190]:
def max_min_currency_vacancy_compensation(vacancy_compensation):
    
    if vacancy_compensation[0:2] == 'от':
        min_vacancy_compensation = int(str_to_digit(vacancy_compensation)[0])
        max_vacancy_compensation = None
    elif vacancy_compensation[0:2] == 'до':
        min_vacancy_compensation = None
        max_vacancy_compensation = int(str_to_digit(vacancy_compensation)[0])
    else:
        min_vacancy_compensation = int(str_to_digit(vacancy_compensation)[0])
        max_vacancy_compensation = int(str_to_digit(vacancy_compensation)[1])
    vacancy_currency = currency(vacancy_compensation)
    return min_vacancy_compensation, max_vacancy_compensation, vacancy_currency

Функция поиска полей в объекте SOUP (поиск по странице запроса), возвращает список словарей с искомыми полями.
Cтруктура словаря:
dict_vacancy = {       
                       'name_vacancy': name_vacancy,                                   # имя вакансии
                       'min_vacancy_compensation': min_vacancy_compensation,           # зарплата от
                       'max_vacancy_compensation': max_vacancy_compensation,           # зарплата до
                       'currency_vacancy_compensation': currency_vacancy_compensation, # валюта зарплаты
                       'link_vacancy': link_vacancy,                                   # ссылка на вакансию
                       'vacancy_employer': vacancy_employer,                           # работодатель
                       'vacancy_address': vacancy_address                              # адрес работодателя

In [192]:
def search_vacancies_soup(soup, list_dict_vacancies):
    job_vacancies = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    for job_vacancy in job_vacancies:
        name_vacancy = job_vacancy.find('a', {'data-qa': 'serp-item__title'}).text
        try:
            vacancy_compensation = job_vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            min_vacancy_compensation, max_vacancy_compensation, currency_vacancy_compensation = max_min_currency_vacancy_compensation(vacancy_compensation)
        except AttributeError:
            vacancy_compensation = ''
            min_vacancy_compensation, max_vacancy_compensation, currency_vacancy_compensation = None, None, None
        link_vacancy = job_vacancy.find('a', {'data-qa': 'serp-item__title'})['href']
        vacancy_employer = job_vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
        vacancy_address = job_vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).text
        dict_vacancy = {'name_vacancy': name_vacancy,
                       'min_vacancy_compensation': min_vacancy_compensation,
                       'max_vacancy_compensation': max_vacancy_compensation,
                       'currency_vacancy_compensation': currency_vacancy_compensation,
                       'link_vacancy': link_vacancy,
                       'vacancy_employer': vacancy_employer,
                       'vacancy_address': vacancy_address
                       }
        list_dict_vacancies.append(dict_vacancy)
    print('parsing page ok!')
    #print(list_dict_vacancies)
    return list_dict_vacancies

Выполнение парсинга

In [193]:
list_dict_vacancies = parser_vacancy_hhru(url_get = url, headers_get = headers, list_dict_vacancies = [])

parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing start!!!
parsing page ok!
parsing stop!!!


In [194]:
DF_vacancies = pd.DataFrame(list_dict_vacancies)

In [195]:
DF_vacancies.head(10)

,name_vacancy,min_vacancy_compensation,max_vacancy_compensation,currency_vacancy_compensation,link_vacancy,vacancy_employer,vacancy_address
0,Data Scientist (команда Коммуникации),350000.0,NaN,руб.,https://krasnoyarsk.hh.ru/vacancy/76096739?fro...,ООО HeadHunter::Analytics/Data Science,"Москва, Алексеевская"
1,Data scientist,200000.0,240000.0,руб.,https://krasnoyarsk.hh.ru/vacancy/74376536?fro...,Астор,Москва
2,Data science,180000.0,210000.0,руб.,https://krasnoyarsk.hh.ru/vacancy/75914060?fro...,7RedLines,Москва
3,Аналитик корпоративных массивов данных (с навы...,NaN,NaN,None,https://krasnoyarsk.hh.ru/vacancy/76385297?fro...,ЭкспрессДеньги,"Москва, Выставочная и еще 2"
4,"Bioinformatics Developer, New Zealand",5000.0,7500.0,USD,https://krasnoyarsk.hh.ru/vacancy/76866591?fro...,ГК ВИЗАВИ Консалт,Москва
5,Global Head of Data Science & Analytics,NaN,550000.0,руб.,https://krasnoyarsk.hh.ru/vacancy/76765163?fro...,Dostavista.ru служба доставки,Москва
6,"Менеджер по обработке данных (SPSS, SAS, Python)",NaN,NaN,None,https://krasnoyarsk.hh.ru/vacancy/76290598?fro...,ORO,Москва
7,"Data scientist, Цены и анализ конкурентов, Мат...",NaN,NaN,None,https://krasnoyarsk.hh.ru/vacancy/76845266?fro...,Ozon Информационные технологии,"Москва, Выставочная и еще 2"
8,Data Scientist / специалист по работе с данными,NaN,NaN,None,https://krasnoyarsk.hh.ru/vacancy/76764137?fro...,"билайн: ИТ, Data, Digital",Москва
9,Стажер Data Scientist,NaN,NaN,None,https://krasnoyarsk.hh.ru/vacancy/76705049?fro...,Сбер для экспертов,Москва


In [199]:
DF_vacancies.to_csv('hh_vacancies_ds.csv', index = False)

In [200]:
df = pd.read_table('hh_vacancies_ds.csv', sep=',')
df

,name_vacancy,min_vacancy_compensation,max_vacancy_compensation,currency_vacancy_compensation,link_vacancy,vacancy_employer,vacancy_address
0,Data Scientist (команда Коммуникации),350000.0,NaN,руб.,https://krasnoyarsk.hh.ru/vacancy/76096739?fro...,ООО HeadHunter::Analytics/Data Science,"Москва, Алексеевская"
1,Data scientist,200000.0,240000.0,руб.,https://krasnoyarsk.hh.ru/vacancy/74376536?fro...,Астор,Москва
2,Data science,180000.0,210000.0,руб.,https://krasnoyarsk.hh.ru/vacancy/75914060?fro...,7RedLines,Москва
3,Аналитик корпоративных массивов данных (с навы...,NaN,NaN,NaN,https://krasnoyarsk.hh.ru/vacancy/76385297?fro...,ЭкспрессДеньги,"Москва, Выставочная и еще 2"
4,"Bioinformatics Developer, New Zealand",5000.0,7500.0,USD,https://krasnoyarsk.hh.ru/vacancy/76866591?fro...,ГК ВИЗАВИ Консалт,Москва
...,...,...,...,...,...,...,...
195,"Доцент кафедры ""Интеллектуальные системы в упр...",70000.0,NaN,руб.,https://krasnoyarsk.hh.ru/vacancy/72008062?fro...,МТУСИ,"Москва, Авиамоторная"
196,PHP-разработчик (myTracker),NaN,NaN,NaN,https://krasnoyarsk.hh.ru/vacancy/75781124?fro...,"VK, ВКонтакте",Москва
197,Лид редакторов,NaN,NaN,NaN,https://krasnoyarsk.hh.ru/vacancy/75786699?fro...,SkillFactory,Москва
198,Методический редактор,NaN,NaN,NaN,https://krasnoyarsk.hh.ru/vacancy/75789661?fro...,SkillFactory,Москва


In [201]:
df['link_vacancy']

0      https://krasnoyarsk.hh.ru/vacancy/76096739?fro...
1      https://krasnoyarsk.hh.ru/vacancy/74376536?fro...
2      https://krasnoyarsk.hh.ru/vacancy/75914060?fro...
3      https://krasnoyarsk.hh.ru/vacancy/76385297?fro...
4      https://krasnoyarsk.hh.ru/vacancy/76866591?fro...
                             ...                        
195    https://krasnoyarsk.hh.ru/vacancy/72008062?fro...
196    https://krasnoyarsk.hh.ru/vacancy/75781124?fro...
197    https://krasnoyarsk.hh.ru/vacancy/75786699?fro...
198    https://krasnoyarsk.hh.ru/vacancy/75789661?fro...
199    https://krasnoyarsk.hh.ru/vacancy/76085457?fro...
Name: link_vacancy, Length: 200, dtype: object